<a href="https://colab.research.google.com/github/madaan/reasoning-tutorial/blob/main/Part_2%20Prompting_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few-shot/0-shot prompting

In [2]:
!pip install openai --quiet
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from tqdm import tqdm
import openai



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00


#### LLM tooling setup


In [21]:
openai.api_key = "NONE"
# you will need a key for the actual openai API

openai.api_base = "http://pitt.lti.cs.cmu.edu:8002/v1"


def call_openai_chatcompletion_api(
    prompt: str,
    engine_name: str,
    temperature: float = 0.0,
    n: int = 1,
    stop: str = None,
    max_tokens: int = 100,
    batch_size: int = 32
):
    """
    Call the OpenAI API to create completions based on the input prompt.

    Parameters:
    - prompt (str): The prompt.
    - engine_name (str, optional): The engine to use for the completion.
    - temperature (float, optional): Sampling temperature for randomness. Defaults to 0.0.
    - n (int, optional): Number of completions to generate. Defaults to 1.
    - stop (str, optional): Token at which the API should stop generating further tokens. Defaults to '\n'.
    - max_tokens (int, optional): Maximum number of tokens in the generated output. Defaults to 100.
    - batch_size (int, optional): Maximum num_completions for each API call. Defaults to 32.

    Returns:
    - list/str: Generated text completions from the API. Returns a list of strings if n > 1, else a single string.
    """
    all_responses = []
    orig_n = n

    messages=[{
        "role": "system",
        "content": "You are a helpful assistant."
    }, {
        "role": "user",
        "content": prompt
    }]

    try:
        while n > 0:
            current_batch_size = min(n, batch_size)

            response = openai.ChatCompletion.create(
                        model=engine_name,
                        messages=messages,
                        temperature=temperature,
                        max_tokens=max_tokens,
                        n=current_batch_size,
                        stop=stop,
                    )

            all_responses.extend([choice["message"]["content"] for choice in response['choices']])

            n -= current_batch_size

        return all_responses if orig_n > 1 else all_responses[0]

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None



### Prompting basics

In [22]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the interactive module
class FewShotPrompting:
    def __init__(self):
        self.textarea = widgets.Textarea(
            value='',
            placeholder='Type your prompt here...',
            description='',
            disabled=False,
            layout=widgets.Layout(height='200px', width='80%'),
            style={'description_width': 'initial', 'font_size': '16px'}
        )
        self.submit_button = widgets.Button(
            description="Submit",
            button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click to get a response',
            icon='',
            layout=widgets.Layout(width='20%')
        )
        self.output = widgets.Output()

        self.submit_button.on_click(self.on_submit)
        display(widgets.VBox([widgets.Label(value="Prompt:"), self.textarea, self.submit_button, self.output]))

    def on_submit(self, button, engine_name = "meta-llama/Llama-2-70b-chat-hf"):
        with self.output:
            clear_output(wait=True)
            prompt = self.textarea.value
            if not prompt:
                print("Please enter a prompt.")
                return

            # Call the OpenAI API
            response = call_openai_chatcompletion_api(prompt, engine_name)
            print(f"Response:\n\n{response}")

# Create an instance to display the interactive module
FewShotPrompting()
print("")

### Interesting applications of prompting

In [26]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def get_response(prompt):
    response = call_openai_chatcompletion_api(prompt, "meta-llama/Llama-2-70b-chat-hf", max_tokens=500)
    return response

def on_button_click(button):
    prompt = text.value
    output_area.value = "Processing... Please wait..."
    response = get_response(prompt)
    output_area.value = f"Response:\n\n{response}"

tasks = {
    "Descriptive Writing": "Describe a serene and peaceful evening on a beach.",
    "Question & Answer": "Explain the concept of quantum entanglement in simple terms.",
    "Translation": "Translate the following English sentence to French: 'Hello, how are you?'",
    "Math Problem Solving": "Solve the quadratic equation x^2 - 5x + 6 = 0.",
    "Code Writing": "Write a Python function that calculates the factorial of a number.",
    "Historical Facts": "Tell me about the Renaissance period.",
}

def on_dropdown_change(change):
    text.value = change['new']

dropdown = widgets.Dropdown(
    options=tasks,
    description='Task:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)
dropdown.observe(on_dropdown_change, names='value')

text = widgets.Textarea(
    value=dropdown.value,
    placeholder='Type your prompt here...',
    layout=widgets.Layout(width='80%', height='100px', margin='20px 0 20px 0'),
    description='',
    disabled=False
)

output_style = {
    'description_width': 'initial',
    'font_size': '16px',
    'font_family': 'Courier New',
    'background_color': '#f7f7f7'
}

output_area = widgets.Textarea(
    value="",
    placeholder='Response will appear here...',
    layout=widgets.Layout(width='80%', height='200px', border='2px solid #0077b6', margin='20px 0 20px 0'),
    style=output_style,
    disabled=True
)

button = widgets.Button(
    description="Get Response",
    button_style='info',
    layout=widgets.Layout(width='150px')
)
button.on_click(on_button_click)

def display_ui():
    display(widgets.VBox([dropdown, text, button, output_area]))

display_ui()
